In [1]:
import os
import glob
import pickle
from datetime import datetime
from pathlib import Path
from tqdm import tqdm
import numpy as np
import pandas as pd
import pandas.api.types as ptypes
import matplotlib.pyplot as plt
from sklearn.preprocessing import QuantileTransformer, LabelEncoder, StandardScaler, MinMaxScaler

In [2]:
data_path = os.path.abspath('../data/favorita-grocery-sales-forecasting')
# set parent directory as the output path
output_path = '../data/favorita-grocery-sales-forecasting'

In [5]:
# No records will be considered outside these bounds
# start_date = datetime(2015, 7, 1)
# end_date = datetime(2017, 4, 1)

# Where training period ends and the validation period begins
# validation_bound = datetime(2016, 7, 1)

start_date = datetime(2017, 1, 1)
end_date = datetime(2017, 4, 1)

validation_bound = datetime(2017, 3, 1)

In [6]:
history_len = 30  # historical scope in time-steps
future_len = 10  # futuristic scope in time-steps

In [7]:
samp_interval = 3  # time-steps

### Attributes configuration

In [8]:
# These are the variables that are known in advance, and will compose the futuristic time-series
known_attrs = ['onpromotion',
               'day_of_week',
               'day_of_month',
               'month',
               'national_holiday',
               'regional_holiday',
               'local_holiday',
               'open'
               ]

# The following set of variables will be considered as static, i.e. containing non-temporal information
# every attribute which is not listed here will be considered as temporal.
static_attrs = ['item_nbr',
                'store_nbr',
                'city',
                'state',
                'store_type',
                'store_cluster',
                'item_family',
                'item_class',
                'perishable',
                ]

# The following set of variables will be considered as categorical.
# The rest of the variables (which are not listed below) will be considered as numeric.
categorical_attrs = ['item_nbr',
                     'store_nbr',
                     'city',
                     'state',
                     'store_type',
                     'store_cluster',
                     'item_family',
                     'item_class',
                     'perishable',
                     'onpromotion',
                     'open',
                     'day_of_week',
                     'month',
                     'national_holiday',
                     'regional_holiday',
                     'local_holiday',
                     ]

In [9]:
target_signal = 'log_sales'

In [10]:
# these will not be included as part of the input data which will end up feeding the model
meta_attrs = ['date', 'combination_id', 'temporal_id', 'unit_sales']

### Data Loading

In [11]:
file_names = [os.path.basename(f) for f in glob.glob(os.path.join(data_path, '*.{}'.format('csv')))]
print(file_names)

['items.csv', 'test.csv', 'sample_submission.csv', 'holidays_events.csv', 'stores.csv', 'transactions.csv', 'train.csv', 'oil.csv']


#### Load the CSV files:

In [12]:
transactions_df = pd.read_csv(os.path.join(data_path, 'transactions.csv'), parse_dates=['date'])
items_df = pd.read_csv(os.path.join(data_path, 'items.csv'), index_col='item_nbr')
oil_df = pd.read_csv(os.path.join(data_path, 'oil.csv'), parse_dates=['date'],index_col='date')
holiday_df = pd.read_csv(os.path.join(data_path, 'holidays_events.csv'), parse_dates=['date'],
                         dtype={'transferred': bool})
stores_df = pd.read_csv(os.path.join(data_path, 'stores.csv'), index_col='store_nbr')


In [13]:
list_data_df = []
data_chunk_iter = pd.read_csv(os.path.join(data_path, 'train.csv'),
                      dtype={'onpromotion': object},
                      index_col='id',
                      parse_dates=['date'], chunksize=100000)

for chunk in data_chunk_iter:
    data_df = chunk.loc[(chunk['date'] >= start_date) & (chunk['date'] <= end_date)]
    list_data_df.append(data_df)

data_df = pd.concat(list_data_df)
print(data_df.shape)

(9436869, 5)


In [14]:
data_df

,date,store_nbr,item_nbr,unit_sales,onpromotion
id,,,,,
101688779,2017-01-01,25,99197,1.0,False
101688780,2017-01-01,25,103665,7.0,False
101688781,2017-01-01,25,105574,1.0,False
101688782,2017-01-01,25,105857,4.0,False
101688783,2017-01-01,25,106716,2.0,False
...,...,...,...,...,...
111125643,2017-04-01,54,2057387,2.0,False
111125644,2017-04-01,54,2058758,1.0,False
111125645,2017-04-01,54,2060793,2.0,False


In [15]:
if ptypes.is_object_dtype(data_df['onpromotion']):
    data_df['onpromotion'] = data_df['onpromotion'] == 'True'

In [16]:
stores_df.rename(columns={'type': 'store_type', 'cluster': 'store_cluster'}, inplace=True)
items_df.rename(columns={'class': 'item_class', 'family': 'item_family'}, inplace=True)
oil_df.rename(columns={'dcoilwtico': 'oil_price'}, inplace=True)
holiday_df.rename(columns={'type': 'holiday_type'}, inplace=True)

In [17]:
# Lose the null records on the raw dataframe representing oil prices
oil_df = oil_df.loc[~oil_df.oil_price.isna()]
oil_df = oil_df.resample('1d').ffill().reset_index()

### Filter, Maniplate & Resample

#### Filter

In [18]:
# have done before
# data_df = data_df.loc[(data_df['date'] >= start_date) & (data_df['date'] <= end_date)]

#### Manipulate

In [19]:
import random

data_df = data_df.assign(combination_id=data_df['store_nbr'].apply(str) + '_' + data_df['item_nbr'].apply(str))

# samples id to avoid big data
n=1000
combination_id_sample = random.sample(list(data_df["combination_id"].unique()), n)
data_df = data_df[data_df["combination_id"].isin(combination_id_sample)] 

# another index can be used to identify the unique combination of (store,product,date)
data_df = data_df.assign(temporal_id=data_df['combination_id'] + '_' + data_df['date'].dt.strftime('%Y-%m-%d'))

In [20]:
data_df

,date,store_nbr,item_nbr,unit_sales,onpromotion,combination_id,temporal_id
id,,,,,,,
101689332,2017-01-01,25,859966,4.0,False,25_859966,25_859966_2017-01-01
101689391,2017-01-01,25,939130,2.0,False,25_939130,25_939130_2017-01-01
101689894,2017-01-01,25,1457229,2.0,False,25_1457229,25_1457229_2017-01-01
101690177,2017-01-01,25,1909755,1.0,True,25_1909755,25_1909755_2017-01-01
101690192,2017-01-01,25,1916579,5.0,False,25_1916579,25_1916579_2017-01-01
...,...,...,...,...,...,...,...
111124997,2017-04-01,54,1243817,5.0,False,54_1243817,54_1243817_2017-04-01
111125010,2017-04-01,54,1306198,6.0,False,54_1306198,54_1306198_2017-04-01
111125025,2017-04-01,54,1328672,16.0,False,54_1328672,54_1328672_2017-04-01


In [21]:
# mark all the existing records as days in which the relevant stores were open
data_df = data_df.assign(open=1)

### Temporal resampling of each combination (1 days interval)

In [22]:
sequence_per_combination = []  # a list to contain all the resampled sequences

# for each combination
for comb_id, comb_df in tqdm(data_df.groupby('combination_id')):
    resamp_seq = comb_df.copy()
    resamp_seq = resamp_seq.set_index('date').resample('1d').last().reset_index()

    resamp_seq['log_sales'] = np.log10(1 + resamp_seq['unit_sales'] + 1e-5)
    # newly generated records are assumed to be days in which the store was not open
    resamp_seq['open'] = resamp_seq['open'].fillna(0)
    # pad with the corresponding information according to the previously available record
    for col in ['store_nbr', 'item_nbr', 'onpromotion']:
        resamp_seq[col] = resamp_seq[col].fillna(method='ffill')

    sequence_per_combination.append(resamp_seq)

# combine all the resampled sequences
data_df = pd.concat(sequence_per_combination, axis=0)

 20%|███████▌                               | 195/1000 [00:00<00:02, 397.09it/s]/home/xyshu/.local/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
100%|██████████████████████████████████████| 1000/1000 [00:02<00:00, 403.29it/s]


### Gathering Information

In [23]:
data_df['day_of_week'] = pd.to_datetime(data_df['date'].values).dayofweek
data_df['day_of_month'] = pd.to_datetime(data_df['date'].values).day
data_df['month'] = pd.to_datetime(data_df['date'].values).month

### Merging with other sources

In [24]:
data_df = data_df.merge(stores_df, how='left', on='store_nbr')
data_df = data_df.merge(items_df, how='left', on='item_nbr')

In [25]:
# we'll ignore holidays that were "transferred"
holiday_df = holiday_df.loc[~holiday_df.transferred]

# National holidays will mark every relevant record (by date)
data_df = data_df.assign(national_holiday=data_df.merge(holiday_df.loc[holiday_df.locale == 'National'],
                                                        on='date', how='left')['description'].fillna('None')
                         )

# Regional holidays will mark every relevant record (by date and state)
data_df = data_df.assign(regional_holiday=data_df.merge(holiday_df.loc[holiday_df.locale == 'Regional'],
                                                        left_on=['date', 'state'],
                                                        right_on=['date', 'locale_name'],
                                                        how='left'
                                                        )['description'].fillna('None')
                         )

# Local holidays will mark every relevant record (by date and city)
data_df = data_df.assign(local_holiday=data_df.merge(holiday_df.loc[holiday_df.locale == 'Local'],
                                                     left_on=['date', 'city'],
                                                     right_on=['date', 'locale_name'],
                                                     how='left'
                                                     )['description'].fillna('None')
                         )

In [26]:
data_df = data_df.merge(transactions_df, how='left', on=['date', 'store_nbr'])
data_df['transactions'] = data_df['transactions'].fillna(-1)

data_df = data_df.merge(oil_df, on='date', how='left')

### Inferring Composition

In [27]:
all_cols = list(data_df.columns)
feature_cols = [col for col in all_cols if col not in meta_attrs]

In [41]:
feature_map = {
    'static_feats_numeric': [col for col in feature_cols if col in static_attrs and col not in categorical_attrs],
    'static_feats_categorical': [col for col in feature_cols if col in static_attrs and col in categorical_attrs],
    'historical_ts_numeric': [col for col in feature_cols if col not in static_attrs and col not in categorical_attrs],
    'historical_ts_categorical': [col for col in feature_cols if col not in static_attrs and col in categorical_attrs],
    'future_ts_numeric': [col for col in feature_cols if col in known_attrs and col not in categorical_attrs],
    'future_ts_categorical': [col for col in feature_cols if col in known_attrs and col in categorical_attrs]
}


feature_map

{'static_feats_numeric': [],
 'static_feats_categorical': ['store_nbr',
  'item_nbr',
  'city',
  'state',
  'store_type',
  'store_cluster',
  'item_family',
  'item_class',
  'perishable'],
 'historical_ts_numeric': ['log_sales',
  'day_of_month',
  'transactions',
  'oil_price'],
 'historical_ts_categorical': ['onpromotion',
  'open',
  'day_of_week',
  'month',
  'national_holiday',
  'regional_holiday',
  'local_holiday'],
 'future_ts_numeric': ['day_of_month'],
 'future_ts_categorical': ['onpromotion',
  'open',
  'day_of_week',
  'month',
  'national_holiday',
  'regional_holiday',
  'local_holiday']}

### Data Scaling

In [29]:
# allocate a dictionary to contain the scaler and encoder objects after fitting them
scalers = {'numeric': dict(), 'categorical': dict()}
# for the categorical variables we would like to keep the cardinalities (how many categories for each variable)
categorical_cardinalities = dict()

In [30]:
# take only the the train time range
only_train = data_df.loc[data_df['date'] < validation_bound]
only_train

,date,store_nbr,item_nbr,unit_sales,onpromotion,combination_id,temporal_id,open,log_sales,day_of_week,...,store_type,store_cluster,item_family,item_class,perishable,national_holiday,regional_holiday,local_holiday,transactions,oil_price
0,2017-01-02,10.0,1098624.0,30.0,False,10_1098624,10_1098624_2017-01-02,1.0,1.491362,0,...,C,15,PERSONAL CARE,4114,0,Traslado Primer dia del ano,None,None,1039,53.75
1,2017-01-03,10.0,1098624.0,19.0,False,10_1098624,10_1098624_2017-01-03,1.0,1.301030,1,...,C,15,PERSONAL CARE,4114,0,None,None,None,1106,52.36
2,2017-01-04,10.0,1098624.0,41.0,False,10_1098624,10_1098624_2017-01-04,1.0,1.623249,2,...,C,15,PERSONAL CARE,4114,0,None,None,None,834,53.26
3,2017-01-05,10.0,1098624.0,30.0,False,10_1098624,10_1098624_2017-01-05,1.0,1.491362,3,...,C,15,PERSONAL CARE,4114,0,None,None,None,842,53.77
4,2017-01-06,10.0,1098624.0,32.0,False,10_1098624,10_1098624_2017-01-06,1.0,1.518514,4,...,C,15,PERSONAL CARE,4114,0,None,None,None,780,53.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83202,2017-02-24,9.0,830797.0,3.0,0.0,9_830797,9_830797_2017-02-24,1.0,0.602061,4,...,B,6,GROCERY I,1074,0,None,None,None,1931,53.99
83203,2017-02-25,9.0,830797.0,6.0,0.0,9_830797,9_830797_2017-02-25,1.0,0.845099,5,...,B,6,GROCERY I,1074,0,None,None,None,2262,53.99
83204,2017-02-26,9.0,830797.0,8.0,0.0,9_830797,9_830797_2017-02-26,1.0,0.954243,6,...,B,6,GROCERY I,1074,0,None,None,None,1740,53.99
83205,2017-02-27,9.0,830797.0,7.0,0.0,9_830797,9_830797_2017-02-27,1.0,0.903091,0,...,B,6,GROCERY I,1074,0,Carnaval,None,None,1812,54.04


In [35]:
only_train.head(20)

,date,store_nbr,item_nbr,unit_sales,onpromotion,combination_id,temporal_id,open,log_sales,day_of_week,...,store_type,store_cluster,item_family,item_class,perishable,national_holiday,regional_holiday,local_holiday,transactions,oil_price
0,2017-01-02,10.0,1098624.0,30.0,False,10_1098624,10_1098624_2017-01-02,1.0,1.491362,0,...,C,15,PERSONAL CARE,4114,0,Traslado Primer dia del ano,None,None,1039,53.75
1,2017-01-03,10.0,1098624.0,19.0,False,10_1098624,10_1098624_2017-01-03,1.0,1.301030,1,...,C,15,PERSONAL CARE,4114,0,None,None,None,1106,52.36
2,2017-01-04,10.0,1098624.0,41.0,False,10_1098624,10_1098624_2017-01-04,1.0,1.623249,2,...,C,15,PERSONAL CARE,4114,0,None,None,None,834,53.26
3,2017-01-05,10.0,1098624.0,30.0,False,10_1098624,10_1098624_2017-01-05,1.0,1.491362,3,...,C,15,PERSONAL CARE,4114,0,None,None,None,842,53.77
4,2017-01-06,10.0,1098624.0,32.0,False,10_1098624,10_1098624_2017-01-06,1.0,1.518514,4,...,C,15,PERSONAL CARE,4114,0,None,None,None,780,53.98
5,2017-01-07,10.0,1098624.0,33.0,False,10_1098624,10_1098624_2017-01-07,1.0,1.531479,5,...,C,15,PERSONAL CARE,4114,0,None,None,None,1027,53.98
6,2017-01-08,10.0,1098624.0,39.0,False,10_1098624,10_1098624_2017-01-08,1.0,1.602060,6,...,C,15,PERSONAL CARE,4114,0,None,None,None,936,53.98
7,2017-01-09,10.0,1098624.0,24.0,False,10_1098624,10_1098624_2017-01-09,1.0,1.397940,0,...,C,15,PERSONAL CARE,4114,0,None,None,None,879,51.95
8,2017-01-10,10.0,1098624.0,24.0,False,10_1098624,10_1098624_2017-01-10,1.0,1.397940,1,...,C,15,PERSONAL CARE,4114,0,None,None,None,881,50.82
9,2017-01-11,10.0,1098624.0,18.0,False,10_1098624,10_1098624_2017-01-11,1.0,1.278754,2,...,C,15,PERSONAL CARE,4114,0,None,None,None,726,52.19


### Fitting the scalers/encoders

In [36]:
for col in tqdm(feature_cols):
    if col in categorical_attrs:
        scalers['categorical'][col] = LabelEncoder().fit(only_train[col].values)
        categorical_cardinalities[col] = only_train[col].nunique()
    else:
        if col in ['log_sales']:
            scalers['numeric'][col] = StandardScaler().fit(only_train[col].values.astype(float).reshape(-1, 1))
        elif col in ['day_of_month']:
            scalers['numeric'][col] = MinMaxScaler().fit(only_train[col].values.astype(float).reshape(-1, 1))
        else:
            scalers['numeric'][col] = QuantileTransformer(n_quantiles=256).fit(
                only_train[col].values.astype(float).reshape(-1, 1))

100%|██████████████████████████████████████████| 20/20 [00:00<00:00, 624.90it/s]


### Transform by Applying Scalers

In [37]:
for col in feature_cols:
    if col in categorical_attrs:
        print(f"categorical_attrs {col}/{len(feature_cols)}")
        le = scalers['categorical'][col]
        # handle cases with unseen keys
        le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
        data_df[col] = data_df[col].apply(lambda x: le_dict.get(x, max(le.transform(le.classes_)) + 1))
        data_df[col] = data_df[col].astype(np.int32)
    else:
        print(f"continous_attrs {col}/{len(feature_cols)}")
        data_df[col] = scalers['numeric'][col].transform(data_df[col].values.reshape(-1, 1)).squeeze()
        # data_df[col] = parallel_transform(scalers['numeric'][col], data_df[col])
        data_df[col] = data_df[col].astype(np.float32)

categorical_attrs store_nbr/20
categorical_attrs item_nbr/20
categorical_attrs onpromotion/20
categorical_attrs open/20
continous_attrs log_sales/20
categorical_attrs day_of_week/20
continous_attrs day_of_month/20
categorical_attrs month/20
categorical_attrs city/20
categorical_attrs state/20
categorical_attrs store_type/20
categorical_attrs store_cluster/20
categorical_attrs item_family/20
categorical_attrs item_class/20
categorical_attrs perishable/20
categorical_attrs national_holiday/20
categorical_attrs regional_holiday/20
categorical_attrs local_holiday/20
continous_attrs transactions/20
continous_attrs oil_price/20


In [38]:
data_df['log_sales'].fillna(0.0, inplace=True)

### Splitting Data

In [39]:
data_sets = {'train': dict(), 'validation': dict(), 'test': dict()}

In [40]:
for combination_id, combination_seq in tqdm(data_df.groupby('combination_id')):

    # take the complete sequence associated with this combination and break it into the relevant periods
    train_subset = combination_seq.loc[combination_seq['date'] < validation_bound]
    num_train_records = len(train_subset)
    validation_subset_len = num_train_records + future_len
    validation_subset = combination_seq.iloc[num_train_records - history_len: validation_subset_len]
    test_subset = combination_seq.iloc[validation_subset_len - history_len:]
    print(train_subset.shape, validation_subset.shape, test_subset.shape)
    
    subsets_dict = {'train': train_subset,
                    'validation': validation_subset,
                    'test': test_subset}

    # for the specific combination we're processing in the current iteration,
    # we'd like to go over each subset separately
    for subset_key, subset_data in subsets_dict.items():
        # sliding window, according to samp_interval skips between adjacent windows
        for i in range(0, len(subset_data), samp_interval):
            # slice includes history period and horizons period
            slc = subset_data.iloc[i: i + history_len + future_len]

            if len(slc) < (history_len + future_len):
                # skip edge cases, where not enough steps are included
                continue

            # meta
            data_sets[subset_key].setdefault('time_index', []).append(slc.iloc[history_len - 1]['date'])
            data_sets[subset_key].setdefault('combination_id', []).append(combination_id)

            # static attributes
            data_sets[subset_key].setdefault('static_feats_numeric', []).append(
                slc.iloc[0][feature_map['static_feats_numeric']].values.astype(np.float32))
            data_sets[subset_key].setdefault('static_feats_categorical', []).append(
                slc.iloc[0][feature_map['static_feats_categorical']].values.astype(np.int32))

            # historical
            data_sets[subset_key].setdefault('historical_ts_numeric', []).append(
                slc.iloc[:history_len][feature_map['historical_ts_numeric']].values.astype(np.float32).reshape(
                    history_len, -1))
            data_sets[subset_key].setdefault('historical_ts_categorical', []).append(
                slc.iloc[:history_len][feature_map['historical_ts_categorical']].values.astype(np.int32).reshape(
                    history_len, -1))

            # futuristic (known)
            data_sets[subset_key].setdefault('future_ts_numeric', []).append(
                slc.iloc[history_len:][feature_map['future_ts_numeric']].values.astype(np.float32).reshape(future_len,
                                                                                                           -1))
            data_sets[subset_key].setdefault('future_ts_categorical', []).append(
                slc.iloc[history_len:][feature_map['future_ts_categorical']].values.astype(np.int32).reshape(future_len,
                                                                                                             -1))

            # target
            data_sets[subset_key].setdefault('target', []).append(
                slc.iloc[history_len:]['log_sales'].values.astype(np.float32))

  2%|▋                                        | 17/1000 [00:00<00:13, 75.47it/s]

(58, 24) (40, 24) (52, 24)
(18, 24) (11, 24) (2, 24)
(52, 24) (40, 24) (51, 24)
(34, 24) (40, 24) (36, 24)
(18, 24) (0, 24) (2, 24)
(47, 24) (40, 24) (43, 24)
(56, 24) (40, 24) (51, 24)
(47, 24) (40, 24) (47, 24)
(40, 24) (40, 24) (45, 24)
(54, 24) (40, 24) (52, 24)
(58, 24) (40, 24) (52, 24)
(29, 24) (0, 24) (39, 24)
(24, 24) (2, 24) (34, 24)
(35, 24) (40, 24) (36, 24)
(54, 24) (40, 24) (52, 24)
(58, 24) (40, 24) (52, 24)
(58, 24) (40, 24) (52, 24)
(34, 24) (40, 24) (39, 24)
(32, 24) (38, 24) (28, 24)
(57, 24) (40, 24) (47, 24)
(2, 24) (3, 24) (3, 24)
(32, 24) (40, 24) (44, 24)
(31, 24) (40, 24) (46, 24)
(14, 24) (14, 24) (6, 24)
(39, 24) (40, 24) (43, 24)
(39, 24) (40, 24) (33, 24)
(42, 24) (40, 24) (41, 24)
(53, 24) (40, 24) (50, 24)
(48, 24) (40, 24) (48, 24)
(56, 24) (40, 24) (51, 24)


  7%|██▋                                     | 67/1000 [00:00<00:09, 103.59it/s]

(57, 24) (40, 24) (50, 24)
(58, 24) (40, 24) (52, 24)
(52, 24) (40, 24) (52, 24)
(1, 24) (1, 24) (1, 24)
(47, 24) (40, 24) (52, 24)
(35, 24) (40, 24) (43, 24)
(29, 24) (0, 24) (41, 24)
(48, 24) (40, 24) (46, 24)
(43, 24) (40, 24) (46, 24)
(45, 24) (40, 24) (43, 24)
(35, 24) (36, 24) (26, 24)
(41, 24) (40, 24) (45, 24)
(44, 24) (40, 24) (47, 24)
(12, 24) (15, 24) (8, 24)
(32, 24) (40, 24) (39, 24)
(51, 24) (40, 24) (43, 24)
(38, 24) (40, 24) (45, 24)
(52, 24) (40, 24) (49, 24)
(17, 24) (13, 24) (3, 24)
(46, 24) (31, 24) (21, 24)
(38, 24) (40, 24) (39, 24)
(55, 24) (40, 24) (51, 24)
(28, 24) (0, 24) (44, 24)
(1, 24) (1, 24) (1, 24)
(55, 24) (40, 24) (50, 24)
(42, 24) (40, 24) (42, 24)
(17, 24) (13, 24) (3, 24)
(51, 24) (40, 24) (50, 24)
(53, 24) (40, 24) (47, 24)
(11, 24) (11, 24) (9, 24)
(19, 24) (11, 24) (1, 24)
(16, 24) (14, 24) (4, 24)
(57, 24) (40, 24) (52, 24)
(55, 24) (40, 24) (50, 24)
(19, 24) (3, 24) (1, 24)
(48, 24) (40, 24) (50, 24)
(55, 24) (40, 24) (51, 24)


  9%|███▋                                    | 92/1000 [00:00<00:08, 108.18it/s]

...

(58, 24) (40, 24) (52, 24)
(57, 24) (40, 24) (52, 24)
(57, 24) (40, 24) (49, 24)
(58, 24) (40, 24) (52, 24)
(52, 24) (40, 24) (49, 24)
(51, 24) (40, 24) (51, 24)
(54, 24) (40, 24) (51, 24)
(43, 24) (40, 24) (44, 24)
(53, 24) (40, 24) (52, 24)
(11, 24) (16, 24) (9, 24)
(15, 24) (15, 24) (5, 24)
(34, 24) (40, 24) (45, 24)
(51, 24) (40, 24) (51, 24)
(44, 24) (40, 24) (44, 24)
(54, 24) (40, 24) (52, 24)
(44, 24) (40, 24) (44, 24)
(47, 24) (40, 24) (49, 24)
(57, 24) (40, 24) (52, 24)


 98%|███████████████████████████████████████▏| 979/1000 [00:10<00:00, 86.28it/s]

(58, 24) (40, 24) (51, 24)
(57, 24) (40, 24) (52, 24)
(56, 24) (34, 24) (24, 24)
(57, 24) (40, 24) (43, 24)
(48, 24) (40, 24) (43, 24)
(25, 24) (0, 24) (39, 24)
(40, 24) (40, 24) (30, 24)
(57, 24) (40, 24) (50, 24)
(13, 24) (17, 24) (7, 24)
(24, 24) (2, 24) (34, 24)
(7, 24) (10, 24) (10, 24)
(31, 24) (40, 24) (43, 24)
(58, 24) (40, 24) (51, 24)
(56, 24) (40, 24) (49, 24)
(30, 24) (40, 24) (37, 24)
(51, 24) (40, 24) (52, 24)
(48, 24) (40, 24) (49, 24)
(51, 24) (40, 24) (51, 24)
(56, 24) (40, 24) (51, 24)
(33, 24) (40, 24) (40, 24)
(56, 24) (40, 24) (52, 24)


100%|███████████████████████████████████████| 1000/1000 [00:11<00:00, 89.50it/s]

(57, 24) (40, 24) (51, 24)
(45, 24) (40, 24) (40, 24)
(56, 24) (40, 24) (52, 24)
(47, 24) (40, 24) (47, 24)
(56, 24) (40, 24) (49, 24)
(58, 24) (40, 24) (52, 24)
(53, 24) (40, 24) (51, 24)
(2, 24) (2, 24) (2, 24)
(51, 24) (40, 24) (52, 24)
(58, 24) (40, 24) (52, 24)
(58, 24) (40, 24) (52, 24)
(56, 24) (40, 24) (49, 24)
(57, 24) (40, 24) (51, 24)


In [40]:
data_df[data_df["combination_id"]=='10_1165987']

,date,store_nbr,item_nbr,unit_sales,onpromotion,combination_id,temporal_id,open,log_sales,day_of_week,...,store_type,store_cluster,item_family,item_class,perishable,national_holiday,regional_holiday,local_holiday,transactions,oil_price
0,2017-01-02,9,398,3.0,0,10_1165987,10_1165987_2017-01-02,1,-0.358983,0,...,2,14,10,13,0,2,0,0,0.217367,0.680392
1,2017-01-03,9,398,1.0,0,10_1165987,10_1165987_2017-01-03,1,-1.161426,1,...,2,14,10,13,0,1,0,0,0.258824,0.209804
2,2017-01-04,9,398,2.0,1,10_1165987,10_1165987_2017-01-04,1,-0.692027,2,...,2,14,10,13,0,1,0,0,0.113122,0.543137
3,2017-01-05,9,398,4.0,1,10_1165987,10_1165987_2017-01-05,1,-0.100654,3,...,2,14,10,13,0,1,0,0,0.117647,0.696078
4,2017-01-06,9,398,11.0,1,10_1165987,10_1165987_2017-01-06,1,0.912861,4,...,2,14,10,13,0,1,0,0,0.091373,0.849020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2017-03-26,9,398,4.0,0,10_1165987,10_1165987_2017-03-26,1,-0.100654,6,...,2,14,10,13,0,1,0,0,0.135472,0.000000
84,2017-03-27,9,398,1.0,0,10_1165987,10_1165987_2017-03-27,1,-1.161426,0,...,2,14,10,13,0,1,0,0,0.110709,0.000000
85,2017-03-28,9,398,1.0,0,10_1165987,10_1165987_2017-03-28,1,-1.161426,1,...,2,14,10,13,0,1,0,0,0.088235,0.000000
87,2017-03-30,9,398,8.0,0,10_1165987,10_1165987_2017-03-30,1,0.579816,3,...,2,14,10,13,0,1,0,0,0.097603,0.000000


In [34]:
# for each set
for set_key in list(data_sets.keys()):
    # for each component in the set
    for arr_key in list(data_sets[set_key].keys()):
        # list of arrays will be concatenated
        if isinstance(data_sets[set_key][arr_key], np.ndarray):
            data_sets[set_key][arr_key] = np.stack(data_sets[set_key][arr_key], axis=0)
        # lists will be transformed into arrays
        else:
            data_sets[set_key][arr_key] = np.array(data_sets[set_key][arr_key])

In [35]:
with open(os.path.join(output_path, 'data.pickle'), 'wb') as f:
    pickle.dump({
        'data_sets': data_sets,
        'feature_map': feature_map,
        'scalers': scalers,
        'categorical_cardinalities': categorical_cardinalities
    }, f, pickle.HIGHEST_PROTOCOL)